In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np
import torch.optim as optim
from matplotlib import pyplot as plt

In [78]:
import numpy as np
import utils
import torch 
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

filename = "/home/wafa/Desktop/study/sketchRNN/aaron_sheep.npz"
load_data = np.load(filename, encoding = 'latin1')
train_set = load_data['train'][:200]
test_set = load_data['test'][:100]

nb_steps = 100
feature_len=5
batch_size = 1
max_seq_len=250

train_set = utils.DataLoader(train_set, batch_size)
test_set = utils.DataLoader(test_set, batch_size)

total images <= max_seq_len is 200
total images <= max_seq_len is 100


In [79]:
class Encoder(nn.Module):
    
    def __init__(self, strokeSize, batchSize,  Nhe, Nhd, Nz):
        super(Encoder, self).__init__()
        self.Nz = Nz
        self.Nhe = Nhe
        self.Nhd = Nhd
        self.cell = nn.LSTM(strokeSize, Nhe//2, 1, bidirectional=True, batch_first=True)
        self.mu = nn.Linear(Nhe, Nz)
        self.sigma = nn.Linear(Nhe, Nz)
        self.h0 = nn.Linear(Nz, Nhd*2) # returns h0 and c0
        #print(Nh)

    def forward(self, x):
        _, (hn, cn) = self.cell(x)
        hn = Variable(torch.cat((hn.data[0],hn.data[1]),1))
        sigma = self.sigma(hn)
        mu = self.mu(hn)
        s = sigma
        sigma = torch.exp( sigma * 0.5)
        eps = Variable(torch.randn(self.Nz))
        z = mu + sigma * eps
        return F.tanh(self.h0(z)), z, mu, s


In [196]:
class Decoder(nn.Module):
    
    def __init__(self, strokeSize, batchSize,  Nhe, Nhd, Nz, Ny):
        super(Decoder, self).__init__()
        self.Nhe = Nhe
        self.Nhd = Nhd
        self.Nz = Nz
        self.batchSize = batchSize
        self.cell = nn.LSTM(strokeSize+Nz, Nhd, 1, batch_first=True)
        self.y = nn.Linear(Nhd, Ny)
    
    def forward(self, x, h0, c0):
        output, (hn, cn) = self.cell(x, (h0, c0))
        #print("avant reshape", output.size())
        #print(output.contiguous().view(-1, self.Nh).size())
        output = output.contiguous().view(-1, self.Nhd)
        #print("after", output.size())
        y = self.y(output)
        return y  
    
    def predict(self, x, h0, c0, z):
        out = []
        tmp = 1
        #h0, z, mu, sigma = self.encoder(x[:, 1:250+1, :])
        #new_input = torch.cat((x[:, :250, :], z.view(self.batchSize, 1, self.Nz).expand(self.batchSize, self.max_seq_len, self.Nz)), 2)
        #h = h0[:,:self.Nhd]
        #c = h0[:,self.Nhd:]
        new_input = torch.cat((x[:, 0, :].contiguous().view(self.batchSize, 1, 5), z.view(self.batchSize, 1, self.Nz)), 2)
        y, (h, c) = self.cell(new_input, (h0, c0))
        output = y.contiguous().view(-1, self.Nhd)
        y = self.y(output)
        
        out.append(y)
        z_pen_logits = y[:, -3:]
        z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr = torch.chunk(y[:, :-3], 6, dim=1)
        z_pi = F.softmax(z_pi)
        z_pen_logits = F.softmax(z_pen_logits)
        z_sigma1 = torch.exp(z_sigma1)
        z_sigma2 = torch.exp(z_sigma2)
        z_corr = F.tanh(z_corr)
        
        #normalize the probs to get 1 
        probs = np.array(list(z_pi.data[0]))
        probs /= probs.sum()
        i = np.random.choice(np.arange(0, 2), p= probs)
        
        mean = [z_mu1.data[0][i], z_mu2.data[0][i]]
        cov = [[z_sigma1.data[0][i] * z_sigma1.data[0][i], z_corr.data[0][i] * z_sigma1.data[0][i] * z_sigma2.data[0][i]], [z_corr.data[0][i] * z_sigma1.data[0][i] * z_sigma2.data[0][i], z_sigma2.data[0][i] * z_sigma2.data[0][i]]]
        
        x = np.random.multivariate_normal(mean, cov, 1)
        
        probs = np.array(list(z_pen_logits.data[0]))
        probs /= probs.sum()
        
        iPen = np.random.choice(np.arange(0, 3), p=probs)
        
        pen = [0, 0, 0]
        pen[iPen] = 1
        
        stroke = [x[0][0], x[0][1]] + pen
        print(stroke)
        
        '''new_input = torch.cat((y, z.view(self.batchSize, 1, self.Nz)), 2)
        for i in range(1,250):
            y, (h, c) = self.cell(new_input, h, c)
            out.append(y)
        return out
        '''

In [197]:
class SketchRNN(nn.Module):
    
    def __init__(self, strokeSize, batchSize, Nhe, Nhd, Nz, Ny, max_seq_len):
        super(SketchRNN, self).__init__()
        self.batchSize = batchSize
        self.Nhe = Nhe
        self.Nhd = Nhd
        self.max_seq_len = max_seq_len
        self.Nz = Nz
        self.encoder = Encoder(strokeSize, batchSize, Nhe, Nhd, Nz)
        self.decoder = Decoder(strokeSize, batchSize, Nhe, Nhd, Nz, Ny)
        
    def forward(self, x):
        # we don't take the inisiale S0 in the encoder so we do the 1:
        h0, z, mu, sigma = self.encoder(x[:, 1:250+1, :])
        #here we take S0
        new_input = torch.cat((x[:, :250, :], z.view(self.batchSize, 1, self.Nz).expand(self.batchSize, self.max_seq_len, self.Nz)), 2)
        y = self.decoder(new_input, h0[:,:self.Nhd], h0[:,self.Nhd:])
        return y, mu, sigma
    
    def predict(self, x):
        
        h0, z, mu, sigma = self.encoder(x[:, 1:250+1, :])
        y = self.decoder.predict(x[:, :250, :], h0[:,:self.Nhd], h0[:,self.Nhd:], z)

        


In [202]:
M = 2
sketchRnn = SketchRNN(5, 1, 256, 512, 128, 6*M+3, 250)
lr=1e-2
for _ in range(10):
    _, x, s = train_set.random_batch()
    x = Variable(torch.from_numpy(x).type(torch.FloatTensor))
    sketchRnn.predict(x)
    

[-0.021102335091033417, 0.615823257749883, 1, 0, 0]
[-0.78079561230249805, 0.014256787788105187, 0, 1, 0]
[0.80993878695683108, -0.76205757172356825, 1, 0, 0]
[1.1633158694604326, -2.2467039218719513, 0, 1, 0]
[2.5294720664366146, -2.352020059199019, 0, 0, 1]
[0.2112516582064129, 1.3520768574843387, 0, 1, 0]
[0.28230208658823797, 0.1717442292336675, 0, 1, 0]
[-0.42966591017080563, 0.77735168226250484, 0, 1, 0]
[0.40359272698325688, 0.79893997270539874, 0, 0, 1]
[0.78488694934939474, -1.2081477565872407, 0, 0, 1]
